# FLYR Data Science Challenge

_Cameron Smith_

June 2018
___

# Content
1. Overview
2. Understand the data
    * 2.1 Import libraries
    * 2.2 Load the data
    * 2.3 Summaries
        * 2.3.1 Airport data
        * 2.3.2 Booking data
        * 2.3.3 Search data
        * 2.3.4 Columns in common
3. Data preparation
    * 3.1 Overview
        * 3.1.1 Existing features
        * 3.1.2 Features to be created
    * 3.2 Existing features
        * 3.2.1 Handle missing values
        * 3.2.2 Convert to correct format
        * 3.2.3 Bucketize cabin variable
        * 3.2.4 Convert currency
    * 3.3 Engineer new features
        * 3.3.1 Distance traveled
        * 3.3.2 Was the airfare booked?
        * 3.3.3 Was the airfare one way?
        * 3.3.4 Depart diff
        * 3.3.5 Various date features
    * 3.4 Convert categorical data to dummy variables
    * 3.5 Scale numerical data
    * 3.6 Drop unneeded features
    * 3.7 Visualize data
4. Modeling
    * 4.1 Build the model
5. Evaluation
    * 5.1 Model performance
    * 5.2 Feature importance
    * 5.3 Future directions

# 1. Overview
I've been asked to perform two tasks: (1) *predict the value of an offered airfare* and (2) *predict which airfares are booked*. I have been provided with three datasets and a useful document that describes the meaning of the variables in each dataset.

# 2. Understand the data
## 2.1 Import required libraries

In [403]:
%%capture
%%sh
pip install requests
pip install currencyconverter
pip install geocoder

In [405]:
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import numpy as np
import currencyconverter
import requests
import geocoder
#from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
#from scipy import stats

%matplotlib inline

## 2.2 Load the data

Now I'll peek at the data I've been given and read it into a form that's easy to work with.

In [356]:
ls

airport_data.csv   currencyconverter/ derp123.ipynb      venv/
booking_data.csv   derp.py            search_data.csv


In [357]:
%% capture
airport_data = pd.read_csv('airport_data.csv')
booking_data = pd.read_csv('booking_data.csv')
search_data = pd.read_csv('search_data.csv')

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 2.2.1 Airport Data

In [358]:
airport_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5823 entries, 0 to 5822
Data columns (total 9 columns):
iata_code         5823 non-null object
city              5823 non-null object
country           5823 non-null object
latitude          5766 non-null float64
longitude         5766 non-null float64
altitude          5690 non-null float64
timezone          5760 non-null float64
dst               5704 non-null object
aggregate_code    5761 non-null float64
dtypes: float64(5), object(4)
memory usage: 409.5+ KB


In [359]:
airport_data.columns

Index(['iata_code', 'city', 'country', 'latitude', 'longitude', 'altitude',
       'timezone', 'dst', 'aggregate_code'],
      dtype='object')

In [360]:
airport_data.head(5)

,iata_code,city,country,latitude,longitude,altitude,timezone,dst,aggregate_code
0,NYC,New York,United States,40.7142,-74.00580,31.0,-5.0,A,1.0
1,QSF,San Francisco,United States,NaN,NaN,NaN,-8.0,NaN,1.0
2,CHI,Chicago,United States,41.8836,-87.63170,596.0,-6.0,A,1.0
3,PAR,Paris,France,48.8564,2.35222,107.0,1.0,E,1.0
4,WAS,Washington,United States,38.8894,-77.03530,25.0,-5.0,A,1.0


### 2.2.2 Booking Data

In [361]:
booking_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16854 entries, 0 to 16853
Data columns (total 25 columns):
booking_id              16854 non-null object
booking_time            16854 non-null object
currency                16854 non-null object
destination             16854 non-null object
booking_user_id         16854 non-null object
language                16854 non-null object
origin                  16854 non-null object
partner_id              16854 non-null int64
passengers              16854 non-null int64
pos                     15504 non-null object
session_id              16854 non-null object
user_agent              13485 non-null object
num_requests            16854 non-null int64
fare                    16854 non-null float64
supplier                16854 non-null int64
cabin_class             16854 non-null object
carrier_1               16854 non-null object
carrier_2               13378 non-null object
flight_num_1            16854 non-null object
flight_num_2        

In [362]:
booking_data.corr()

,partner_id,passengers,num_requests,fare,supplier
partner_id,1.000000,0.019721,-0.099739,0.030768,-0.707887
passengers,0.019721,1.000000,-0.006152,-0.014235,-0.027118
num_requests,-0.099739,-0.006152,1.000000,0.045988,0.166444
fare,0.030768,-0.014235,0.045988,1.000000,-0.035824
supplier,-0.707887,-0.027118,0.166444,-0.035824,1.000000


In [363]:
booking_data.columns

Index(['booking_id', 'booking_time', 'currency', 'destination',
       'booking_user_id', 'language', 'origin', 'partner_id', 'passengers',
       'pos', 'session_id', 'user_agent', 'num_requests', 'fare', 'supplier',
       'cabin_class', 'carrier_1', 'carrier_2', 'flight_num_1', 'flight_num_2',
       'departure_datetime_1', 'arrival_datetime_1', 'departure_datetime_2',
       'arrival_datetime_2', 'itinerary_id'],
      dtype='object')

In [364]:
booking_data.head(5)

,booking_id,booking_time,currency,destination,booking_user_id,language,origin,partner_id,passengers,pos,...,cabin_class,carrier_1,carrier_2,flight_num_1,flight_num_2,departure_datetime_1,arrival_datetime_1,departure_datetime_2,arrival_datetime_2,itinerary_id
0,7ec12305-ba84-419a-8fc0-6aaf78d9abb2,2017-02-01 00:00:46,USD,SJD,36195bda-ec40-4907-9428-c4fe11bd87fa,en,LAX,91,1,US,...,economy,AA,AA,AA237,AA237,2017-05-30T08:35:00,2017-05-30T11:58:00,2017-06-03T13:00:00,2017-06-03T14:36:00,c53826cf-1fba-42a6-a765-941628e1901e
1,5eb6e42e-bcdc-4b0a-8b3a-744b4ef1d805,2017-02-01 00:01:03,USD,LAX,025ee32b-2a0b-4987-b275-893f148ab259,en,CMH,91,1,US,...,economy,AA,NaN,AA659,NaN,2017-03-29T16:35:00,2017-03-29T18:53:00,NaN,NaN,2e3afcd8-7d56-4b49-a71b-f85c23e1b807
2,d3db456d-538d-4f98-a2a0-d6bcedf3b459,2017-02-01 00:01:04,USD,MCO,f69465d7-cdb2-4dd0-98ad-6362423a8dc3,en,EGE,91,1,US,...,economy,UA; UA,UA; UA,UA4475; UA564,UA1191; UA1252,2017-02-10T07:52:00; 2017-02-10T11:00:00,2017-02-10T09:03:00; 2017-02-10T16:31:00,2017-02-13T08:47:00; 2017-02-13T11:40:00,2017-02-13T10:30:00; 2017-02-13T13:41:00,619b0f7c-825f-47b1-a191-300862feff80
3,b9133358-7e81-4aea-bce8-8c79823c4ac8,2017-02-01 00:01:12,USD,LHR,36ba8a8c-8c47-4dc6-bd70-0b3ae071c8cf,en,PHX,91,2,US,...,economy,DL; DL,DL; DL,DL5852; DL4414,DL11; DL1516,2017-05-10T13:13:00; 2017-05-10T18:30:00,2017-05-10T14:45:00; 2017-05-11T12:55:00,2017-05-28T12:55:00; 2017-05-28T17:50:00,2017-05-28T16:05:00; 2017-05-28T19:11:00,37a44686-8347-4c72-8690-c9432349f9a6
4,4699db09-65c2-492f-9aa6-c9e7734a60c1,2017-02-01 00:01:25,USD,LHR,b1a66917-ca3e-457e-8bb2-79b6cd86ca7c,en,SFO,91,1,US,...,economy,TP; TP; TP,TP; TP; TP,TP8052; TP218; TP352,TP353; TP209; TP8051,2017-03-06T08:00:00; 2017-03-06T20:00:00; 2017...,2017-03-06T16:35:00; 2017-03-07T07:15:00; 2017...,2017-03-09T06:00:00; 2017-03-09T16:50:00; 2017...,2017-03-09T08:35:00; 2017-03-09T20:15:00; 2017...,b84a4eca-fd5f-4794-afaa-e1d0e7231537


### 2.2.3 Search Data

In [365]:
search_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1277062 entries, 0 to 1277061
Data columns (total 25 columns):
search_id               1277062 non-null object
search_time             1277062 non-null object
currency                1277062 non-null object
destination             1277062 non-null object
search_user_id          1277062 non-null object
language                1277062 non-null object
origin                  1277062 non-null object
partner_id              1277062 non-null int64
passengers              1277062 non-null int64
pos                     1271955 non-null object
session_id              1277062 non-null object
user_agent              1268917 non-null object
num_requests            1277062 non-null int64
fare                    1277062 non-null float64
supplier                1277062 non-null int64
cabin_class             1277062 non-null object
carrier_1               1277062 non-null object
carrier_2               1042662 non-null object
flight_num_1            12

In [366]:
search_data.shape

(1277062, 25)

In [367]:
search_data.columns

Index(['search_id', 'search_time', 'currency', 'destination', 'search_user_id',
       'language', 'origin', 'partner_id', 'passengers', 'pos', 'session_id',
       'user_agent', 'num_requests', 'fare', 'supplier', 'cabin_class',
       'carrier_1', 'carrier_2', 'flight_num_1', 'flight_num_2',
       'departure_datetime_1', 'arrival_datetime_1', 'departure_datetime_2',
       'arrival_datetime_2', 'itinerary_id'],
      dtype='object')

In [368]:
search_data.head(5)

,search_id,search_time,currency,destination,search_user_id,language,origin,partner_id,passengers,pos,...,cabin_class,carrier_1,carrier_2,flight_num_1,flight_num_2,departure_datetime_1,arrival_datetime_1,departure_datetime_2,arrival_datetime_2,itinerary_id
0,02c6c2ad-325b-4155-bb03-89badc92ea84,2017-02-01 00:00:07,USD,MIA,NBYNv9utORVjlf604rUgXL/eFz6Tmu2NyygDWTFsM5OOWH...,en,LAX,294,1,US,...,economy,AA,AA,AA1088,AA1147,2017-04-07T09:50:00-07:00,2017-04-07T17:56:00-04:00,2017-04-16T21:30:00-04:00,2017-04-17T00:33:00-07:00,294f363b-86f0-4e67-abe9-05221dadb8b8
1,0280e382-d24e-4138-bb20-8d236c0bb02f,2017-02-01 00:00:32,ISK,VCE,c2ea8bd4-40ff-4b8b-a42f-ada9b02ef963,en,REK,58,1,IS,...,Economy,FI; IB,NaN,FI540; IB5120,NaN,2017-06-10T01:05:00; 2017-06-10T14:50:00,2017-06-10T06:20:00; 2017-06-10T16:40:00,NaN,NaN,d9592a05-c850-4abd-9cf1-9fe8343d556c
2,c8903983-3eb9-4f0f-8f87-64b7e5dd0692,2017-02-01 00:00:39,USD,SJD,36195bda-ec40-4907-9428-c4fe11bd87fa,en,LAX,91,1,US,...,economy,AA,AA,AA237,AA237,2017-05-30T08:35:00,2017-05-30T11:58:00,2017-06-03T13:00:00,2017-06-03T14:36:00,c53826cf-1fba-42a6-a765-941628e1901e
3,b1d666c8-d570-47ec-bc46-26ee5332620f,2017-02-01 00:00:48,USD,LHR,36ba8a8c-8c47-4dc6-bd70-0b3ae071c8cf,en,PHX,91,2,US,...,economy,DL; DL,DL; DL,DL5852; DL4414,DL11; DL1516,2017-05-10T13:13:00; 2017-05-10T18:30:00,2017-05-10T14:45:00; 2017-05-11T12:55:00,2017-05-28T12:55:00; 2017-05-28T17:50:00,2017-05-28T16:05:00; 2017-05-28T19:11:00,37a44686-8347-4c72-8690-c9432349f9a6
4,020521d7-ea85-49dd-97e2-1ae662f432a9,2017-02-01 00:00:48,ISK,BFS,cc24b343-7e53-48b4-9bdb-092a49381665,is,REK,58,2,IS,...,Economy,FI; FR,FR; U2,FI470; FR1274,FR1138; U28507,2017-04-12T07:35:00; 2017-04-12T18:10:00,2017-04-12T11:35:00; 2017-04-12T19:35:00,2017-04-19T20:00:00; 2017-04-20T06:30:00,2017-04-19T21:20:00; 2017-04-20T08:45:00,52248983-0449-411f-9794-88c7bc098faa


It looks like `airport_data` contains information about various airports (listed as `destination` or `origin` in the other data files), `booking_data` contains information about successful bookings, and `search_data` contains information about searches. It's also apparent that many of the entries in `booking_data` are connected to the entries in `search_data` by their `itinerary_id` column. I'll use this information in the feature engineering stage.

### 2.2.4  Columns in common
It seems like `booking_data` and `search_data` have most of their columns in common. So I'll check the symmetric difference of the sets (`booking_data` $\bigoplus$ `search_data`) to see what items I'll have to work on independently. For the rest, I'll simply loop over both `search_data` and `booking_data`.

#### Unique

In [369]:
%pprint
set(booking_data.columns) ^ set(search_data.columns)

Pretty printing has been turned OFF


{'search_user_id', 'search_time', 'booking_user_id', 'booking_time', 'booking_id', 'search_id'}

#### In common
And here are the columns that the datasets have in common, which I'll start cleaning:

In [370]:
set(booking_data.columns) & set(search_data.columns)

{'departure_datetime_2', 'arrival_datetime_1', 'user_agent', 'fare', 'itinerary_id', 'carrier_1', 'cabin_class', 'flight_num_1', 'destination', 'session_id', 'supplier', 'carrier_2', 'currency', 'partner_id', 'departure_datetime_1', 'pos', 'origin', 'arrival_datetime_2', 'passengers', 'language', 'num_requests', 'flight_num_2'}

# 3. Data preparation
## 3.1 Overview of features
Different columns will need to be cleaned up in different ways. I'll make some predictions about which columns to keep, which to modify, and which to drop. I'm going to make my life easier by not trying to calculate the total flight time, which would require de-localizing the various departure/arrival times. Instead I'll only concern myself with features generated from the first departure time. Trying to mix together both one-way and return flights is a bit of a headache, because all one-way flights will have missing values for the return column. In the table below, "departure" means the first flight the passenger boards during their trip.

### 3.1.1 Existing features


| column               | data type    | comments                | fate    |
| ---------------------|:------------:|:-----------------------:|--------:|
| departure_datetime_1 | datetime     | extract features        | drop             |
| departure_datetime_2 | datetime     | extract features        | drop             |
| arrival_datetime_1   | datetime     | extract features        | drop             |
| arrival_datetime_2   | datetime     | extract features        | drop             |
| cabin_class          | categorical  | convert to "buckets"    | dummy variables  |
| carrier_1 *            | categorical  | lots of carriers        | drop            |
| carrier_2 *            | categorical  | lots of carriers        | drop            |
| currency *             | categorical  | use for USD conversion  | drop            |
| origin *               | categorical  | lots of origins         | drop            |
| destination *          | categorical  | lots of destinations    | drop            |
| fare                 | discrete int | dependent variable      | convert to USD   |
| flght_num_1          | categorical  | ID                      | drop             |
| flight_num_2         | categorical  | ID                      | drop             |
| itinerary_id         | categorical  | useful in engineering phase | drop         |
| language             | categorical  | almost all searches are in "en" | drop |
| num_requests         | discrete int |                         | min-max scaled   |
| partner_id *           | categorical  | lots of partners        | drop            |
| passengers           | discrete int |                         | min-max scaled   |
| pos *                  | categorical  | lots of points-of-sale  | drop            |
| session_id           | categorical  | ID data. Not useful     | drop             |
| supplier *             | categorical  | lots of suppliers       | drop            |
| user_agent †          | categorical  | requires lots of feat. engineering | drop  |
|                      |              |                         |                  |
| booking_id           | categorical  | ID. useful in eng phase | drop             |
| booking_time         | datetime     | convert to "buckets"    | dummy variables  |
| booking_user_id      | categorical  |                         | drop             |
| search_id            | categorical  |                         | drop             |
| search_time          | datetime     | convert to "buckets"    | dummy variables  |
| search_user_id       | categorical  |                         | drop             |

\* Ideally, these categorical variables would be preserved as dummy variables. However, these variables take on a lot of different values, which means the machine algorithms I'll use later on will be a lot slower. So I'm avoiding that pain by simply dropping them.

† I could extract information from this feature, but I don't think it'll be worth the effort. I could return to this if my model's accuracy is not high enough.


### 3.1.3 Features to be created

| new feature (column) | data type      | comments                                 |
| ---------------------|:--------------:|-----------------------------------------:|
| distance_traveled    | continuous int | haversine distance                       |
| one_way              | binary         | was the flight one-way?                  |
| was_airfare_booked   | binary         | dependent variable for task (2)         |
| booking_depart_diff  | datetime       | time between booking and departure       |
| search_depart_diff   | datetime       | time between search and departure        |
| time_till_xmas * | datetime       | time between departure and next holiday  |
| time_from_xmas * | datetime       | time between departure and last holiday  |
| time_till_ny * | datetime       | time between departure and next holiday  |
| time_from_ny * | datetime       | time between departure and last holiday  |
| time_till_tgiving * | datetime       | time between departure and next holiday  |
| time_from_tgiving * | datetime       | time between departure and last holiday  |
| time_till_cny * | datetime       | time between departure and next holiday  |
| time_from_cny * | datetime       | time between departure and last holiday  |
| departure_weekday          | categorical    | was this a weekday - will become a dummy variable |
| departure_day_of_week          | categorical    | the day of the week - will become a dummy variable |
| departure_week_of_year          | categorical    | the week of the year - will become a dummy variable |

\* popular holidays = Christmas, Thanksgiving, New Years, Chinese New Year. I could endlessly engineer date features and even do a little research into other Holidays (Eid, Diwali, Golden Week, etc.) but that might take a lot of time. So I'm limiting my "holidays" to just these three.

## 3.2 Existing features
### 3.2.1 Handle missing values

In [371]:
# List column names with missing values for "booking" and "search" data.
booking_ = dict(booking_data.isnull().sum())
search_ = dict(search_data.isnull().sum())
[(x, booking_[x]) for x in booking_ if booking_[x] > 0] + \
[(x, search_[x]) for x in search_ if search_[x] > 0]

[('pos', 1350), ('user_agent', 3369), ('carrier_2', 3476), ('flight_num_2', 3476), ('departure_datetime_2', 3476), ('arrival_datetime_2', 3476), ('pos', 5107), ('user_agent', 8145), ('carrier_2', 234400), ('flight_num_2', 234400), ('departure_datetime_2', 234400), ('arrival_datetime_2', 234400), ('itinerary_id', 1173941)]

Luckily, these are all columns that are going to be dropped, so they don't need to be considered anymore.

In [372]:
# List column names with missing values for "airport" data.
airport_ = dict(airport_data.isnull().sum())
dict([(x, airport_[x]) for x in airport_ if airport_[x] > 0])

{'latitude': 57, 'longitude': 57, 'altitude': 133, 'timezone': 63, 'dst': 119, 'aggregate_code': 62}

Latitude and longitude are the only columns from this database that I'm going to use that also have missing values. Luckily, information about the city and country exists for every entry. I can use this information to reconstruct the missing coordinate data. This will match cities, not airports, but should be close enough.

In [373]:
# Sort the rows with missing coordinate data into a temporary separate dataframe
# There are 57 missing values.
airport_data_1, airport_data_2 = airport_data.sort_values(by=['longitude'])[:-57], \
     airport_data.sort_values(by=['longitude'])[-57:]

# Define a function to fetch geolocations
def handle_geo(loc, pos):
    """
    This API does not always return a value, so retry several times.
    """
    attempts = 0
    while attempts < 20:
        try:
            return geocoder.google(loc).latlng[pos]
        except:
            attempts += 1
    return None

# Generate temporary column
airport_data_2['location'] = airport_data_2['city'] + ', ' + airport_data_2['country']

# Fetch coordinate data from temporary column
airport_data_2['latitude'] = airport_data_2['location']. \
        apply(lambda x: handle_geo(x, 0))
airport_data_2['longitude'] = airport_data_2['location']. \
        apply(lambda x: handle_geo(x, 1))

# Drop temporary column
airport_data_2 = airport_data_2.drop(columns=['location'])

# Recombine datasets
airport_data = pd.concat([airport_data_1, airport_data_2])

# Drop any rows in case null values crept through
airport_data = airport_data.dropna(subset=['latitude', 'longitude'])

In [374]:
# Drop missing values from airport_data
airport_data = airport_data.dropna(subset=['latitude', 'longitude'])

### 3.2.2 Convert to correct format
Most of the date data is not in date format. This is because the date columns represent the departure/arrival times for all flights on an itinerary, not just one. I decided earlier on to only concern myself with only three date variables: `booking_time`, `search_time`, and the departure time of the very first flight: `departure_datetime_1`.

In [375]:
# Convert booking_time and search_time
booking_data['booking_time'] = pd.to_datetime(booking_data['booking_time'])
search_data['search_time'] = pd.to_datetime(search_data['search_time'])

In [376]:
# Select the first departure time. Convert to date. Coerce errors to NaT
for db in [booking_data, search_data]:
    db['departure_datetime_1'] = pd.to_datetime(db['departure_datetime_1'] \
            .apply(lambda x: str(x).split(';')[0]), errors='coerce')
    
# Confirm that not too many null values were created:
dict(booking_data.isnull().sum())['departure_datetime_1'], \
dict(search_data.isnull().sum())['departure_datetime_1']

(10, 43)

In [377]:
# Drop rows with missing data in departure_datetime_1
booking_data = booking_data.dropna(subset=['departure_datetime_1'])
search_data = search_data.dropna(subset=['departure_datetime_1'])

### 3.2.3 Bucketize cabin variable
I'll start off by handling the various entries in the `cabin` column.

In [378]:
set(booking_data.cabin_class.unique()) & set(search_data.cabin_class.unique())

{'premium_economy', 'Premium Economy', 'COACH', 'first', 'mixed', 'business', 'First', 'Business', 'First Class', 'Economy', 'economy'}

According to [this website](https://www.cheapair.com/help/flights/what-are-the-different-classes-of-service-on-a-plane/), the class of an airline ticket can be divided into four buckets of increasing price/comfort, so I'll modify the data to reflect that (I had to guess about "mixed"):

In [379]:
def cabin_buckets(row):
    if row in ['economy', 'Economy', 'COACH', 'mixed']:
        return 'economy'
    if row in ['premium_economy', 'Premium Economy']:
        return 'premium_economy'
    if row in ['business', 'Business']:
        return 'business'
    if row in ['First', 'First Class']:
        return 'first'

for db in [booking_data, search_data]:
    db['cabin_class'] = db['cabin_class'].apply(lambda x: cabin_buckets(x))

### 3.2.4 Convert fare
The fare information (which is the dependent variable) seems to be in a variety of different currencies. This will cause problems, so let's convert it all to USD. I'll make use of a third-party package to do that.

In [380]:
set(booking_data.currency.unique()) & set(search_data.currency.unique())

{'MYR', 'NZD', 'AUD', 'ZAR', 'HUF', 'CHF', 'CNY', 'PHP', 'AED', 'EUR', 'THB', 'CZK', 'MXN', 'JPY', 'HKD', 'KRW', 'GBP', 'NOK', 'SGD', 'USD', 'DKK', 'CAD', 'ISK', 'SEK'}

In [381]:
import currencyconverter

from currency_converter import CurrencyConverter
c = CurrencyConverter()

# A wrapper function to handle errors.
def convert_wrap(fare, from_, to_):
    try:
        return c.convert(fare, from_, to_)
    except:
        return None
    
# It's annoying to transform this third-party module into something that can be
# easily vectorized, so I'll cheat and create a new column, called fare_map,
# which will tell us what to multiply the "fare" column by in order to get
# the fare in USD. This is trading I/O and CPU time for RAM -- a trade I'm
# happy to make.

currency_mapping = {}
for currency in list(booking_data.currency.unique()) + list(search_data.currency.unique()):
    currency_mapping[currency] = convert_wrap(1, currency, 'USD')
    
for db in [booking_data, search_data]:
    db['fare_map'] = db['currency'].apply(lambda x: currency_mapping[x])

for db in [booking_data, search_data]:
    db['fare'] = db['fare'] * db['fare_map']
    db = db.drop(columns=['fare_map'])
    
# Confirm that not too many null values were created:
dict(booking_data.isnull().sum())['fare'], \
dict(search_data.isnull().sum())['fare']

(41, 917)

In [382]:
# Drop rows with missing data in fare
booking_data = booking_data.dropna(subset=['fare'])
search_data = search_data.dropna(subset=['fare'])

## 3.3 Engineer new features
### 3.3.1 Distance traveled
Now it's time to start engineering features. Let's figure out how far the plane traveled by using the Haversine distance between the origin and destination airports.

In [383]:
# Create a mapping between the airport code and the coordinates.
airport_lat_coordinates = pd.Series(airport_data.latitude.values,index=airport_data.iata_code).to_dict()
airport_lon_coordinates = pd.Series(airport_data.longitude.values,index=airport_data.iata_code).to_dict()

Now let's insert this information into `search_data` and `booking_data` for each flight. I'll make some temporary columns and then compute distance. 

In [384]:
%%capture
# Define some helper functions.
def map_lat_coords(x):
    try:
        return airport_lat_coordinates[x]
    except:
        return None

def map_lon_coords(x):
    try:
        return airport_lon_coordinates[x]
    except:
        return None

# Create some temporary columns with coordinate data.
for db in [booking_data, search_data]:
    db['origin_lat'] = db['origin'].apply(map_lat_coords)
    db['origin_lon'] = db['origin'].apply(map_lon_coords)
    db['destination_lat'] = db['destination'].apply(map_lat_coords)
    db['destination_lon'] = db['destination'].apply(map_lon_coords)
    
# Define a Haversine distance function.
# Surprisingly, numpy does not have this!
from numpy import radians, cos, sin, arcsin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great distance between two points 
    on the earth from their longitude and latitude.
    Returns the distance in kilometers.
    """
    # Convert degrees to radians. 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine distance formula.
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * arcsin(sqrt(a))
    r = 6371 # Radius of earth in kilometers. Miles = 3956.
    return c * r

# Use the columns to calculate the distance traveled and drop temporary columns.
for db in [booking_data, search_data]:
    db['distance_traveled'] = haversine(db['origin_lon'], \
            db['origin_lat'], db['destination_lon'], db['destination_lat'])
    
search_data = search_data.drop(columns=['origin_lon', 'origin_lat', \
        'destination_lon', 'destination_lat'])
booking_data = booking_data.drop(columns=['origin_lon', 'origin_lat', \
        'destination_lon', 'destination_lat'])

In [385]:
# Confirm that there are not too many missing entries
# This is dependent upon the Geopy API getting coordinates
# for every city.
dict(search_data.isnull().sum())['distance_traveled'], \
dict(booking_data.isnull().sum())['distance_traveled']

(449, 4)

In [386]:
booking_data = booking_data.dropna(subset=['distance_traveled'])
search_data = search_data.dropna(subset=['distance_traveled'])

### 3.3.2 Was the airfare booked?

Goal (2) was to predict which airfares are booked. Because there's no `was_airfare_booked` column in the dataset, I'll have to engineer this dependent variable.

In [387]:
# Read the entries into a dict for faster lookup time: dict = O(1), list = O(n).
booked_flight = {}
for flight in booking_data['itinerary_id']:
    booked_flight[flight] = 1

# Function to check whether an entry is in the dict. If not, return 0.
def was_booked(id):
    return booked_flight.get(id, 0)

search_data['was_airfare_booked'] = search_data['itinerary_id']. \
        apply(lambda x: was_booked(x))

### 3.3.3 Was the flight one-way?
If a row has a null value for `flight_num_2`, then it is safe to assume that there was no return flight, and therefore the flight was one-way.

In [388]:
def one_way(x):
    if pd.notnull(x):
        return 0
    return 1

for db in [booking_data, search_data]:
    db['one_way_flight'] = db['flight_num_2'].apply(lambda x: one_way(x))

### 3.3.4 Depart diff
I imagine that flights which are booked earlier will be cheaper. Searches that happen later might correspond to more likely bookings (desperation).

In [389]:
search_data['depart_diff'] = \
        search_data['departure_datetime_1'] - search_data['search_time']
booking_data['depart_diff'] = \
        booking_data['departure_datetime_1'] - booking_data['booking_time']

I also want to know the times between the departures and various holidays. This variable is likely to be non-linear for `search_data`, because flights with very small or negative depart_diffs probably weren't booked, as well as searches from very far out. Let's see if there's a non-linear relationship: **[plot this]**

### 3.3.5 Various date features

In [390]:
for df in [booking_data, search_data]:
    df['departure_dayofweek'] = df['departure_datetime_1'].dt.dayofweek
    df['departure_weekday'] = df['departure_datetime_1'].dt.weekday
    df['departure_weekofyear'] = df['departure_datetime_1'].dt.weekofyear
    df['departure_dayofyear'] = df['departure_datetime_1'].dt.dayofyear

In [391]:
# HOLIDAYS NOT INCLUDED

# import datetime
# def time_till_holiday(time, holiday):
#     # Select the smallest non-negative time.
#     possible_diffs = [day - time for day in holiday]
#     return min([diff > datetime.timedelta(0) for diff in possible_diffs])

# import datetime
# def time_from_holiday(time, holiday):
#     # Select the smallest non-negative time.
#     possible_diffs = [day - time for day in holiday]
#     return min([diff > datetime.timedelta(0) for diff in possible_diffs])

# xmas_times = [pd.Timestamp('2016-25-12'), pd.Timestamp('2017-25-12'), pd.Timestamp('2018-25-12')]
# ny_times = [pd.Timestamp('2016-01-01'), pd.Timestamp('2017-01-01'), pd.Timestamp('2018-01-01')]
# tgiving_times = [pd.Timestamp('2016-11-24'), pd.Timestamp('2017-11-23'), pd.Timestamp('2018-11-22')]
# cny_times = [pd.Timestamp('2016-02-08'), pd.Timestamp('2017-01-28'), pd.Timestamp('2018-02-16')]


# for df in [booking_data, search_data]:
#     df['departure_time_till_xmas'] = df['departure_datetime_1']. \
#             apply(lambda x: time_till_holiday(x, xmas_times))
#     df['departure_time_from_xmas'] = df['departure_datetime_1']
#     df['departure_time_till_ny'] = df['departure_datetime_1']
#     df['departure_time_from_ny'] = df['departure_datetime_1']
#     df['departure_time_till_tgiving'] = df['departure_datetime_1']
#     df['departure_time_from_tgiving'] = df['departure_datetime_1']
#     df['departure_time_till_cny'] = df['departure_datetime_1']
#     df['departure_time_from_cny'] = df['departure_datetime_1']

# booking_data.head(10)

## 3.4 Convert variables
Let's drop the columns we don't need:

In [392]:
booking_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16799 entries, 0 to 16853
Data columns (total 33 columns):
booking_id              16799 non-null object
booking_time            16799 non-null datetime64[ns]
currency                16799 non-null object
destination             16799 non-null object
booking_user_id         16799 non-null object
language                16799 non-null object
origin                  16799 non-null object
partner_id              16799 non-null int64
passengers              16799 non-null int64
pos                     15461 non-null object
session_id              16799 non-null object
user_agent              13457 non-null object
num_requests            16799 non-null int64
fare                    16799 non-null float64
supplier                16799 non-null int64
cabin_class             16755 non-null object
carrier_1               16799 non-null object
carrier_2               13349 non-null object
flight_num_1            16799 non-null object
flight_num_2

In [393]:
%%capture
booking_data = booking_data.drop(columns=['booking_id', 'booking_user_id', \
            'pos', 'user_agent', 'num_requests', 'supplier', 'flight_num_1', \
            'flight_num_2', 'currency', 'carrier_1', 'carrier_2', 'partner_id', \
            'arrival_datetime_1', 'arrival_datetime_2', 'departure_datetime_2', \
            'origin', 'destination', 'fare_map', 'itinerary_id', 'session_id', \
            'language', 'booking_time', 'departure_datetime_1'])

search_data = search_data.drop(columns=['search_id', 'search_user_id', \
            'pos', 'user_agent', 'num_requests', 'supplier', 'flight_num_1', \
            'flight_num_2', 'currency', 'carrier_1', 'carrier_2', 'partner_id', \
            'arrival_datetime_1', 'arrival_datetime_2', 'departure_datetime_2', \
            'origin', 'destination', 'fare_map', 'itinerary_id', 'session_id', \
            'language', 'search_time', 'departure_datetime_1'])

### 3.4.1 Convert categorical data to dummy variables

In [398]:
# for db in [booking_data, search_data]:
for variable in ['cabin_class', 'one_way_flight', 'departure_dayofweek', \
                 'departure_weekday', 'departure_weekofyear']:
    class_dummy = pd.get_dummies(search_data[variable], prefix=variable)
    search_data = search_data.merge(class_dummy, left_index=True, right_index=True)
    search_data = search_data.drop(columns=variable)
    
for variable in ['cabin_class', 'one_way_flight', 'departure_dayofweek', \
                 'departure_weekday', 'departure_weekofyear']:
    class_dummy = pd.get_dummies(booking_data[variable], prefix=variable)
    booking_data = booking_data.merge(class_dummy, left_index=True, right_index=True)
    booking_data = booking_data.drop(columns=variable)

### 3.4.1 Convert timedelta to numeric

In [399]:
booking_data['depart_diff'] = booking_data['depart_diff'].astype(int)
search_data['depart_diff'] = search_data['depart_diff'].astype(int)

In [400]:
booking_data.head(10)

,passengers,fare,distance_traveled,depart_diff,departure_dayofyear,cabin_class_business_x,cabin_class_economy_x,cabin_class_first_x,cabin_class_premium_economy_x,one_way_flight_0_x,...,departure_weekofyear_43_y,departure_weekofyear_44_y,departure_weekofyear_45_y,departure_weekofyear_46_y,departure_weekofyear_47_y,departure_weekofyear_48_y,departure_weekofyear_49_y,departure_weekofyear_50_y,departure_weekofyear_51_y,departure_weekofyear_52_y
0,1,286.7900,1468.102596,10226054000000000,150,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,192.2000,3204.066376,4898037000000000,88,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,615.6000,2655.453559,805856000000000,41,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2,507.4600,8464.632796,8514708000000000,130,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,888.8600,8615.468303,2879915000000000,65,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,3,352.6000,834.965181,9869006000000000,146,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,1,395.6400,1530.956220,1976753000000000,54,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,1,187.4300,1475.127551,1234710000000000,46,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,3,376.6000,1669.789075,4424249000000000,83,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,2,190.9656,1105.264012,8113927000000000,125,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## 3.5 Scale numerical data

In [408]:
# No need to scale passengers.
for variable in ['fare', 'distance_traveled', 'depart_diff']:
    scaler = StandardScaler()
    booking_data[variable] = scaler.fit_transform(booking_data[[variable]])

## 3.7 Visualize data

# 4. Modeling

## 4.1 Build the model

# 5. Evaluation

## 5.1 Model performance

## 5.2 Feature importance

## 5.3 Future directions

In [261]:
search_data.columns, booking_data.columns

(Index(['search_id', 'search_time', 'currency', 'destination', 'search_user_id',
        'language', 'origin', 'partner_id', 'passengers', 'pos', 'session_id',
        'user_agent', 'num_requests', 'fare', 'supplier', 'cabin_class',
        'carrier_1', 'carrier_2', 'flight_num_1', 'flight_num_2',
        'departure_datetime_1', 'arrival_datetime_1', 'departure_datetime_2',
        'arrival_datetime_2', 'itinerary_id', 'fare_map', 'distance_traveled',
        'was_airfare_booked'],
       dtype='object'),
 Index(['booking_id', 'booking_time', 'currency', 'destination',
        'booking_user_id', 'language', 'origin', 'partner_id', 'passengers',
        'pos', 'session_id', 'user_agent', 'num_requests', 'fare', 'supplier',
        'cabin_class', 'carrier_1', 'carrier_2', 'flight_num_1', 'flight_num_2',
        'departure_datetime_1', 'arrival_datetime_1', 'departure_datetime_2',
        'arrival_datetime_2', 'itinerary_id', 'fare_map', 'distance_traveled'],
       dtype='object'))

In [263]:
%%capture
booking_data = booking_data.drop(columns=['booking_id', 'booking_user_id', 'partner_id'])

search_data = search_data.drop(columns=['search_id', 'search_user_id', 'partner_id'])

for db in [booking_data, search_data]:
    db = db.drop(columns=[\
                         'pos', 'user_agent', 'num_requests', 'supplier', 'flight_num_1', \
        'flight_num_2', 'currency', 'carrier_1', 'carrier_2'
                         ])
    
search_data.columns, booking_data.columns

KeyError: "labels ['booking_id' 'booking_user_id' 'partner_id'] not contained in axis"

## Prepare the data for ML Algorithms
Now I need to convert all of the categorical variables into numerical variables. These "dummy variables" add extra compute time, so to make things simple, I've dropped two variables that have a lot of different members: `currency` and `carrier` even though they might be relevant for predicting the outcome.

Some of the other features require modification. It's not fair to compare the cost of a flight with 5 people to a flight with only 1, so I'm going to create a `fare_per_passenger` column that divides `fare` by `passengers` and drop the `fare` column